In [ ]:
%matplotlib inline
import scipy.special, cmath
import random, math, pylab, os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pylab
import mpl_toolkits.mplot3d

# Bosonic statistics: path integral treatement

## MCMC sampling of permutations

In [ ]:
N = 3
statistics = {}
L = range(N)
nsteps = 10000
for step in range(nsteps):
    i = random.randint(0, N - 1)
    j = random.randint(0, N - 1)
    L[i], L[j] = L[j], L[i]
    if tuple(L) in statistics: 
        statistics[tuple(L)] += 1
    else:
        statistics[tuple(L)] = 1
    #print L
    #print range(N)
    #print

for item in statistics:
    print item, statistics[item]

## MCMC sampling for 256 bosonic particles in a harmonic trap

In [ ]:
import random, math, pylab, mpl_toolkits.mplot3d
 
def levy_harmonic_path(k, beta):
    xk = tuple([random.gauss(0.0, 1.0 / math.sqrt(2.0 *
                math.tanh(k * beta / 2.0))) for d in range(3)])
    x = [xk]
    for j in range(1, k):
        Upsilon_1 = (1.0 / math.tanh(beta) +
                     1.0 / math.tanh((k - j) * beta))
        Upsilon_2 = [x[j - 1][d] / math.sinh(beta) + xk[d] /
                     math.sinh((k - j) * beta) for d in range(3)]
        x_mean = [Upsilon_2[d] / Upsilon_1 for d in range(3)]
        sigma = 1.0 / math.sqrt(Upsilon_1)
        dummy = [random.gauss(x_mean[d], sigma) for d in range(3)]
        x.append(tuple(dummy))
    return x
 
def rho_harm(x, xp, beta):
    Upsilon_1 = sum((x[d] + xp[d]) ** 2 / 4.0 *
                    math.tanh(beta / 2.0) for d in range(3))
    Upsilon_2 = sum((x[d] - xp[d]) ** 2 / 4.0 /
                    math.tanh(beta / 2.0) for d in range(3))
    return math.exp(- Upsilon_1 - Upsilon_2)

N = 256
T_star = 0.6
beta = 1.0 / (T_star * N ** (1.0 / 3.0))
nsteps = 40000
positions = {}

# initialize the positions of the particles with Levy paths of length 1
for j in range(N):
    a = levy_harmonic_path(1, beta)
    positions[a[0]] = a[0]
    
for step in range(nsteps):
    # select a random particle...
    boson_a = random.choice(positions.keys())
    perm_cycle = []
    # ... and determine the cycle it belongs to
    while True:
        perm_cycle.append(boson_a)
        boson_b = positions.pop(boson_a)
        if boson_b == perm_cycle[0]:
            break
        else:
            boson_a = boson_b
    # measure the length of this cycle
    k = len(perm_cycle)
    # and resample this cycle, as a Levy path of length k at temperature beta
    perm_cycle = levy_harmonic_path(k, beta)
    # update the positions dictionary
    positions[perm_cycle[-1]] = perm_cycle[0]
    for j in range(len(perm_cycle) - 1):
        positions[perm_cycle[j]] = perm_cycle[j + 1]
        
    # Now sample the permutations, by trying to insert a new transposition
    # Define the candidate transposition
    a_1 = random.choice(positions.keys())
    b_1 = positions.pop(a_1)
    a_2 = random.choice(positions.keys())
    b_2 = positions.pop(a_2)
    # Calculate old and new weights
    weight_new = rho_harm(a_1, b_2, beta) * rho_harm(a_2, b_1, beta)
    weight_old = rho_harm(a_1, b_1, beta) * rho_harm(a_2, b_2, beta)
    # perform Metropololis
    if random.uniform(0.0, 1.0) < weight_new / weight_old:
        positions[a_1] = b_2
        positions[a_2] = b_1
    else:
        positions[a_1] = b_1
        positions[a_2] = b_2

In [ ]:
%matplotlib notebook

fig = pylab.figure()
ax = mpl_toolkits.mplot3d.axes3d.Axes3D(fig)
ax.set_aspect('equal')
list_colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
n_colors = len(list_colors)
dict_colors = {}
i_color = 0
# find and plot permutation cycles:
while positions:
    x, y, z = [], [], []
    starting_boson = positions.keys()[0]
    boson_old = starting_boson
    while True:
        x.append(boson_old[0])
        y.append(boson_old[1])
        z.append(boson_old[2])
        boson_new = positions.pop(boson_old)
        if boson_new == starting_boson:
            break
        else:
            boson_old = boson_new
    len_cycle = len(x)
    if len_cycle > 2:
        x.append(x[0])
        y.append(y[0])
        z.append(z[0])
    if len_cycle in dict_colors:
        color = dict_colors[len_cycle]
        ax.plot(x, y, z, color + '+-', lw=0.75)
    else:
        color = list_colors[i_color]
        i_color = (i_color + 1) % n_colors
        dict_colors[len_cycle] = color
        ax.plot(x, y, z, color + '+-', label='k=%i' % len_cycle, lw=0.75)
    
# finalize plot
pylab.title('$N=%i$, $T^*=%s$' % (N, T_star))
pylab.legend()
ax.set_xlabel('$x$', fontsize=16)
ax.set_ylabel('$y$', fontsize=16)
ax.set_zlabel('$z$', fontsize=16)
ax.set_xlim3d([-8, 8])
ax.set_ylim3d([-8, 8])
ax.set_zlim3d([-8, 8])
pylab.savefig('snapshot_bosons_3d_N%04i_Tstar%04.2f.png' % (N, T_star))
pylab.show()

# Analytic approach and direct sampling

## Cycle length distribution for uniform permutations: MCMC calculation

In [ ]:
%matplotlib inline
N = 20
stats = [0] * (N + 1)
L = range(N)
nsteps = 1000000
for step in range(nsteps):
    # sample the permutations
    i = random.randint(0, N - 1)
    j = random.randint(0, N - 1)
    L[i], L[j] = L[j], L[i]
    # once we have done enough steps, do a measurement
    # of cycle length statistics
    if step % 100 == 0: 
        cycle_dict = {}
        for k in range(N):
            cycle_dict[k] = L[k]
        while cycle_dict != {}:
            starting_element = cycle_dict.keys()[0]
            cycle_length = 0
            old_element = starting_element
            while True:
                cycle_length += 1
                new_element = cycle_dict.pop(old_element)
                if new_element == starting_element:
                    break
                else:
                    old_element = new_element
            stats[cycle_length] += 1

In [ ]:
pylab.plot(range(1, N + 1), stats[1:], 'b-')
pylab.plot(np.arange(1, N + 1), stats[1] / np.arange(1, N+1), 'rv--')
pylab.xlabel(r'cycle length $k$', fontsize=16)
pylab.ylabel('cycle probability $\pi_k$', fontsize=16)
pylab.title('Equiprobable permutations: \n Cycle length distribution')
pylab.xlim(0, 20)
pylab.show()
pylab.close()

# Canonic recursion for bosonic partition function

In [ ]:
def z(k, beta):
    return 1.0 / (1.0 - math.exp(- k * beta)) ** 3

def canonic_recursion(N, beta):
    Z = [1.0]
    for M in range(1, N + 1):
        Z.append(sum(Z[k] * z(M - k, beta) for k in range(M)) / M)
    return Z

## From last week: numerical estimate of $Z$ in the bounded trap model

In [ ]:
Energy = [0.0] + [1.0] * 3 + [2.0] * 6 + [3.0] * 10 + [4.0] * 15

def get_statistics(beta):
    #beta = 1.0
    n_states = 0
    Z = 0.0
    N0_mean = 0.0
    E_mean = 0.0

    for s_0 in range(35):
        for s_1 in range(s_0, 35):
            for s_2 in range(s_1, 35):
                for s_3 in range(s_2, 35):
                    for s_4 in range(s_3, 35):
                        n_states += 1
                        state = [s_0, s_1, s_2, s_3, s_4]
                        E = sum(Energy[s] for s in state)
                        Z += math.exp(-beta * E)
                        E_mean += E * math.exp(-beta * E)
                        N0_mean += state.count(0) * math.exp(-beta * E)
    return (n_states, Z, E_mean / Z / 5.0, N0_mean / Z / 5.0)

## Comparison of analytic vs numeric result for the partition function

In [ ]:
print canonic_recursion(5, 2.0)
print get_statistics(2.0)

## Cycle length distribution for weights of permutation given by single-particle partition function at $k \beta$

In [ ]:
N = 256
T_star = 0.6
T =  N ** (1.0 / 3.0) * T_star
beta = 1.0 / T
Z = canonic_recursion(N, beta)
pi_k = [(z(k, beta) * Z[N - k] / Z[-1]) / float(N) for k in range(1, N + 1)]

In [ ]:
# graphics output
pylab.plot(range(1, N + 1), pi_k, 'b-', lw=2.5)
pylab.ylim(0.0, 0.01)
pylab.xlabel('cycle length $k$', fontsize=16)
pylab.ylabel('cycle probability $\pi_k$', fontsize=16)
pylab.title('Ideal bosons: \n Cycle length distribution ($N=%i$, $T^*=%s$)' % (N, T_star), fontsize=16)
pylab.savefig('plot-prob_cycle_length.png')

# EXERCISE : implement the calculation of the condensate fraction

## Condensate fraction

In [ ]:
def z(k, beta):
    return 1.0 / (1.0 - math.exp(- k * beta)) ** 3

def get_condensate_fraction(N, beta):
    # TO IMPLEMENT

## Test: plot the results for different temperatures

In [ ]:
N = 256
finesse = 50
fracs = []
for T_star in np.linspace(0.1, 1.0, num=finesse):
    T =  N ** (1.0 / 3.0) * T_star
    beta = 1.0 / T
    fracs.append(get_condensate_fraction(N, beta)) 

In [ ]:
# graphics output
pylab.plot(np.linspace(0.1, 1.0, num=finesse), fracs, 'b-', lw=2.5)
#pylab.ylim(0.0, 0.01)
pylab.xlabel('$T^*$', fontsize=16)
pylab.ylabel('$<N_0>$', fontsize=16)
pylab.title('Ideal bosons: \n Condensate fraction ($N=%i$)' % (N), fontsize=16)
pylab.savefig('plot-prob_cycle_length.png')

# Direct sampling

In [ ]:
def z(k, beta):
    return (1.0 - math.exp(- k * beta)) ** (-3)

def canonic_recursion(N, beta):
    Z = [1.0]
    for M in range(1, N + 1):
        Z.append(sum(Z[k] * z(M - k, beta) for k in range(M)) / M)
    return Z

def make_pi_list(Z, M):
    pi_list = [0.0] + [z(k, beta) * Z[M - k] / Z[M] / M for k  in range(1, M + 1)]
    pi_cumulative = [0.0]
    for k in range(1, M + 1):
        pi_cumulative.append(pi_cumulative[k - 1] + pi_list[k])
    return pi_cumulative

def naive_tower_sample(pi_cumulative):
    eta = random.uniform(0.0, 1.0)
    for k in range(len(pi_cumulative)):
        if eta < pi_cumulative[k]:
            break
    return k

def levy_harmonic_path(dtau, N): 
    beta = N * dtau
    x_N = random.gauss(0.0, 1.0 / math.sqrt(2.0 * math.tanh(beta / 2.0)))
    x = [x_N]
    for k in range(1, N):
        dtau_prime = (N - k) * dtau
        Upsilon_1 = 1.0 / math.tanh(dtau) + 1.0 / math.tanh(dtau_prime)
        Upsilon_2 = x[k - 1] / math.sinh(dtau) + x_N / math.sinh(dtau_prime)
        x_mean = Upsilon_2 / Upsilon_1
        sigma = 1.0 / math.sqrt(Upsilon_1)
        x.append(random.gauss(x_mean, sigma))
    return x

# EXERCISE : implement get_direct_bec_config

In [ ]:
def get_direct_bec_config(N, beta):
    # TO IMPLEMENT

## Test: plot the results for different temperatures

In [ ]:
#Number of particles
N = 512
# Relative temperature
T_star = 0.2
T =  N ** (1.0 / 3.0) * T_star
beta = 1.0 / T

x_config, y_config, z_config = get_direct_bec_config(N, beta)

In [ ]:
%matplotlib notebook

fig = pylab.figure()
ax = mpl_toolkits.mplot3d.axes3d.Axes3D(fig)
ax.set_aspect('equal')
# finalize plot
pylab.title('$N=%i$, $T^*=%s$' % (N, T_star))
ax.plot(x_config, y_config, z_config, 'bo', lw=0.75)
ax.set_xlabel('$x$', fontsize=16)
ax.set_ylabel('$y$', fontsize=16)
ax.set_zlabel('$z$', fontsize=16)
ax.set_xlim3d([-8, 8])
ax.set_ylim3d([-8, 8])
ax.set_zlim3d([-8, 8])
pylab.legend()
pylab.savefig('snapshot_bosons_3d_N%04i_Tstar%04.2f.png' % (N, T_star))
pylab.show()